# LaMa로 만든 흑백 복원 이미지를 DDColor로 추론만 함

### 1. 라이브러리 설치

In [ ]:
!pip install modelscope

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.4 MB/s eta 0:00:00


In [ ]:
!pip install addict

In [ ]:
!pip install dlib==19.24.2
!pip install lmdb==1.4.1
!pip install numpy==1.24.3
!pip install opencv_python==4.7.0.72
!pip install Pillow==10.1.0
!pip install PyYAML==6.0.1
!pip install Requests==2.31.0
!pip install scipy==1.9.1
!pip install timm==0.9.2
!pip install torch>=1.7
!pip install torchvision
!pip install tqdm==4.65.0
!pip install wandb==0.15.5
!pip install scikit-image==0.22.0
!pip install tensorboard
!pip install huggingface_hub
!pip install ipykernel
!pip install matplotlib

In [ ]:
!pip install datasets==2.16.0

In [ ]:
!pip install oss2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.3 MB/s eta 0:00:00
  Created wheel for oss2: filename=oss2-2.19.1-py3-none-any.whl size=123943 sha256=ec3fdef3b210aa50efb1998e2ac569026b9ee8535fe995cb57dd2819f957d632
  Stored in directory: /root/.cache/pip/wheels/7d/1b/c6/dbf318597d350b50a94408a23a56b7349e706d61e0a06a6407
  Created wheel for aliyun-python-sdk-core: filename=aliyun_python_sdk_core-2.16.0-py3-none-any.whl size=535316 sha256=978c8f629ebf6625abe523d877de89e8b22fd1e61216ce9394a465ad10ac92e7
  Stored in directory: /root/.cache/pi

In [ ]:
!pip install sortedcontainers

### 2. 만들어놓은 흑백 복원된 이미지 불러오기

In [ ]:
import zipfile
import os

def extract_zip(zip_file_path, extract_to_path):
    """
    ZIP 파일을 지정된 디렉토리에 추출합니다.

    :param zip_file_path: ZIP 파일의 경로
    :param extract_to_path: 추출할 디렉토리 경로
    """
    try:
        # ZIP 파일 열기
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # 디렉토리가 존재하지 않으면 생성
            if not os.path.exists(extract_to_path):
                os.makedirs(extract_to_path)

            # ZIP 파일 추출
            zip_ref.extractall(extract_to_path)
            print(f"ZIP 파일이 {extract_to_path}에 성공적으로 추출되었습니다.")
    except zipfile.BadZipFile:
        print("유효하지 않은 ZIP 파일입니다.")
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")

# 사용 예시
zip_file = "/content/baseline4.zip"  # ZIP 파일 경로
output_dir = "output_directory"  # 추출할 디렉토리 경로
extract_zip(zip_file, output_dir)


ZIP 파일이 output_directory에 성공적으로 추출되었습니다.


In [ ]:
import os

def get_image_paths(folder_path, extensions=("jpg", "jpeg", "png", "bmp", "gif")):
    """
    지정된 폴더에서 이미지 파일의 경로를 추출합니다.

    :param folder_path: 이미지가 있는 폴더 경로
    :param extensions: 추출할 이미지 파일의 확장자 (기본값: jpg, jpeg, png, bmp, gif)
    :return: 이미지 경로 리스트
    """
    image_paths = []
    try:
        for file_name in os.listdir(folder_path):
            # 파일 경로 생성
            file_path = os.path.join(folder_path, file_name)

            # 파일이 이미지인지 확인
            if os.path.isfile(file_path) and file_name.lower().endswith(extensions):
                image_paths.append(file_path)

        return image_paths
    except Exception as e:
        print(f"오류가 발생했습니다: {e}")
        return []

# 사용 예시
folder_path = "/content/output_directory"  # 이미지가 있는 폴더 경로
image_paths = get_image_paths(folder_path)



##### <에러>
##### /usr/local/lib/python3.10/dist-packages/modelscope/fileio/format/jsonplus.py 의 import simplejson as json 이걸 import json as simplejson이거로

### 3. ddcolor로 추론

In [ ]:
import cv2
from modelscope.outputs import OutputKeys
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

img_colorization = pipeline(Tasks.image_colorization, model='damo/cv_ddcolor_image-colorization')
for path in image_paths:
  result = img_colorization(path)
  name = os.path.basename(path)
  cv2.imwrite(f'{name}.png', result[OutputKeys.OUTPUT_IMG])

2024-12-02 12:36:18,954 - modelscope - WARNING - Model revision not specified, use revision: v1.02
2024-12-02 12:36:19,237 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/damo/cv_ddcolor_image-colorization
2024-12-02 12:36:19,239 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/damo/cv_ddcolor_image-colorization.
2024-12-02 12:36:19,243 - modelscope - INFO - initialize model from /root/.cache/modelscope/hub/damo/cv_ddcolor_image-colorization
/usr/local/lib/python3.10/dist-packages/modelscope/models/cv/image_colorization/ddcolor/ddcolor_for_image_colorization.py:114: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the defa